# SolarMED model

Model that integrates all the component models and subproblems into one interface.

This notebook is used to develop and test it

In [9]:
from pathlib import Path
import time
import datetime
import numpy as np
import pandas as pd
from iapws import IAPWS97 as w_props
import hjson
from loguru import logger

# Visualization packages
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

from solarmed_modeling.utils import data_preprocessing, data_conditioning

# auto reload modules
%load_ext autoreload
%autoreload 2

logger.disable("phd_visualizations.utils.units")

# Paths definition
output_path: Path = Path("../../docs/models/attachments")
data_path: Path = Path("../data")

date_str: str = "20230511" # '20230630'
filename_process_data = f'{date_str}_solarMED.csv'
filename_process_data2 = f'{date_str}_MED.csv'

# Available data to test
# data/calibration/20230807_aquasol.csv
# data/calibration/20230707_20230710_datos_tanques.csv
# Nextcloud/Juanmi_MED_PSA/EURECAT/data/20231030_solarMED.csv

sample_rate = '400s'
sample_rate_numeric = int(sample_rate[:-1])

# Resample figures using plotly_resampler
resample_figures: bool = False

# Parameters
cost_w: float = 3 # €/m³, cost of water
cost_e: float = 0.05 # €/kWh, cost of electricity


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Pre-processing

In [10]:
data_paths = [data_path / f"datasets/{filename_process_data}", data_path / f"datasets/{filename_process_data2}"]

with open( data_path / "variables_config.hjson") as f:
    vars_config = hjson.load(f)
    
with open(data_path/"plot_config.hjson") as f:
    plot_config = hjson.load(f)
    
# Load data and preprocess data
df = data_preprocessing(data_paths, vars_config, sample_rate_key=sample_rate)

# Condition data
df = data_conditioning(df, cost_w=cost_w, cost_e=cost_e, sample_rate_numeric=sample_rate_numeric, vars_config=vars_config)


/workspaces/solarMED-modeling/src/solarmed_modeling/utils/__init__.py:135: DtypeWarning:

Columns (42,50,66,73,75) have mixed types. Specify dtype option on import or set low_memory=False.

/workspaces/solarMED-modeling/src/solarmed_modeling/utils/__init__.py:164: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



### Test visualization

In [3]:
# df.to_csv('data/dev.csv')


In [11]:
# Update plot config
with open( data_path / "plot_config.hjson") as f:
    plot_config = hjson.load(f)

fig = experimental_results_plot(plot_config, df, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)


## Evaluation

In [19]:
# Test model
import math
from solarmed_modeling.solar_med import SolarMED

logger.enable("solarmed_modeling")

span = math.ceil(600 / sample_rate_numeric) # 600 s
idx_start = np.max([span, 2]) # idx_start-1 should at least be one 
idx_end = len(df)
df_mod = pd.DataFrame()

# Initialize model
model = SolarMED(
    resolution_mode='constant-water-props',
    use_models=True,
    use_finite_state_machine=True,
    sample_time=sample_rate_numeric,
    
    # Initial states
    ## Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[idx_start], df['Tts_h_m'].iloc[idx_start], df['Tts_h_b'].iloc[idx_start]], 
    Tts_c=[df['Tts_c_t'].iloc[idx_start], df['Tts_c_m'].iloc[idx_start], df['Tts_c_b'].iloc[idx_start]],
    
    ## Solar field
    Tsf_in_ant=df['Tsf_in'].iloc[idx_start-span:idx_start].values,
    qsf_ant=df['qsf'].iloc[idx_start-span:idx_start].values,
    
    # cost_w = 3, # €/m³ 
    # cost_e = 0.05, # €/kWhe,
)


2024-11-14 08:12:30.920 | INFO     | solarmed_modeling.solar_med:init_matlab_engine:1447 - MATLAB engine initialized
2024-11-14 08:12:30.924 | INFO     | solarmed_modeling.solar_med:model_post_init:560 - 
        SolarMED model initialized with: 
            - Evaluating models: True
            - Evaluating finite state machines: True
            - Resolution mode: constant-water-props
            - On limits violation policy: clip
            - Sample time: 400.0 s
            - MED actuators: ['med_brine_pump', 'med_feed_pump', 'med_distillate_pump', 'med_cooling_pump', 'med_heatsource_pump']
            - Solar field actuators: ['sf_pump']
            - Thermal storage actuators: ['ts_src_pump']
            - Model parameters: ModelParameters(sf=ModelParameters(beta=0.0436396, H=13.676448551722462, gamma=0.1), ts=ModelParameters(UA_h=(0.0069818, 0.00584034, 0.03041486), V_h=(5.94771006, 4.87661781, 2.19737023), UA_c=(0.01396848, 0.0001, 0.02286885), V_c=(5.33410037, 7.56470594, 0.9

### Model exporting/importing


In [20]:
# Save model configuration
model_config = model.model_dump_configuration()

# Export model instance and recreate a new instance
dumped_model_instance: dict = model.dump_instance()
# display(dumped_model_instance)
model: SolarMED = SolarMED(**dumped_model_instance)


2024-11-14 08:12:33.252 | INFO     | solarmed_modeling.solar_med:init_matlab_engine:1447 - MATLAB engine initialized
2024-11-14 08:12:33.256 | INFO     | solarmed_modeling.solar_med:model_post_init:560 - 
        SolarMED model initialized with: 
            - Evaluating models: True
            - Evaluating finite state machines: True
            - Resolution mode: constant-water-props
            - On limits violation policy: clip
            - Sample time: 400.0 s
            - MED actuators: ['med_brine_pump', 'med_feed_pump', 'med_distillate_pump', 'med_cooling_pump', 'med_heatsource_pump']
            - Solar field actuators: ['sf_pump']
            - Thermal storage actuators: ['ts_src_pump']
            - Model parameters: ModelParameters(sf=ModelParameters(beta=0.0436396, H=13.676448551722462, gamma=0.1), ts=ModelParameters(UA_h=(0.0069818, 0.00584034, 0.03041486), V_h=(5.94771006, 4.87661781, 2.19737023), UA_c=(0.01396848, 0.0001, 0.02286885), V_c=(5.33410037, 7.56470594, 0.9

### Model evaluation

In [21]:
df_mod = model.to_dataframe(df_mod)

# Run model
for idx in range(idx_start+1, idx_end):
    ds = df.iloc[idx]
    
    # logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
        
    model.step(
        # Decision variables
        ## MED
        qmed_s=ds['qmed_s'],
        qmed_f=ds['qmed_f'],
        Tmed_s_in=ds['Tmed_s_in'],
        Tmed_c_out=ds['Tmed_c_out'],
        ## Thermal storage
        qts_src=ds['qhx_s'],
        ## Solar field
        qsf=ds['qsf'],
        
        med_vacuum_state=2,
                
        # Environment variables
        Tmed_c_in=ds['Tmed_c_in'],
        Tamb=ds['Tamb'],
        I=ds['I'],
    )
    
    logger.info(f"Finished Iteration {idx} / {idx_end} - {df.index[idx]:%H:%M:%S}, elapsed time: {time.time()-start_time:.2f} seconds.")

    df_mod = model.to_dataframe(df_mod)


2024-11-14 08:12:35.659 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:51 - (qmed_s_sp) Value 0.00 is less than 30.00 -> 0.0
2024-11-14 08:12:35.661 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:51 - (qmed_f_sp) Value 0.00 is less than 5.00 -> 0.0
2024-11-14 08:12:35.661 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:51 - (Tmed_s_in_sp) Value 48.60 is less than 60.00 -> 0.0
2024-11-14 08:12:35.662 | DEBUG    | solarmed_modeling.data_validation:within_range_or_min_or_max:73 - (Tmed_c_out_sp) Value 18.68 is less than 18.80 -> 18.80
2024-11-14 08:12:35.663 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:51 - (qts_src_sp) Value 0.00 is less than 0.95 -> 0.0
2024-11-14 08:12:35.664 | INFO     | solarmed_modeling.fsms:check_elapsed_samples:278 - [SF-TS] Still recirculating_ts cooldown, 1/89991 to complete
2024-11-14 08:12:35.665 | DEBUG    | solarmed_modeling.fsms:inform_exit_state

2024-11-14 08:12:35.740 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:51 - (qmed_c) Value 0.00 is less than 8.00 -> 0.0
2024-11-14 08:12:35.742 | INFO     | __main__:<module>:30 - Finished Iteration 8 / 89 - 08:20:00, elapsed time: 0.01 seconds.
2024-11-14 08:12:35.747 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:51 - (qmed_s_sp) Value 0.00 is less than 30.00 -> 0.0
2024-11-14 08:12:35.748 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:51 - (qmed_f_sp) Value 0.00 is less than 5.00 -> 0.0
2024-11-14 08:12:35.749 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:51 - (Tmed_s_in_sp) Value 46.37 is less than 60.00 -> 0.0
2024-11-14 08:12:35.749 | DEBUG    | solarmed_modeling.data_validation:within_range_or_min_or_max:73 - (Tmed_c_out_sp) Value 19.63 is less than 19.86 -> 19.86
2024-11-14 08:12:35.750 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:51 - 

In [22]:
# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx<idx_end-1 else idx_end] # idx_start-1 because now we are adding one element after the initialization

# Update plot config
with open( data_path / "plot_config_validation.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)


In [ ]:
from solarmed_modeling.metrics import calculate_metrics
from solarmed_modeling.solar_med.utils import out_var_ids

metrics = calculate_metrics(df_mod[out_var_ids].values, 
                            df.iloc[idx_start:][out_var_ids].values)

metrics


{'ITAE': np.float64(258666.5659640071),
 'ISE': np.float64(51172.54786231996),
 'IAE': np.float64(5075.691060150004),
 'RMSE': np.float64(8.168096095364142),
 'MAE': np.float64(6.617589387418519),
 'MSE': np.float64(66.71779382310295),
 'R2': np.float64(0.4040454605609911)}

In [56]:
# Save figure
# save_figure(
#     figure_name=f"SolarMED_validation_{df.index[0].strftime('%Y%m%d')}", 
#     figure_path=output_path,
#     fig=fig, formats=('png', 'html'), 
#     width=fig.layout.width, height=fig.layout.height, scale=2
# )


### Model evaluation using `evaluate_model`

In [ ]:
from solarmed_modeling.solar_med.utils import evaluate_model
from solarmed_modeling.solar_med import ModelParameters, FixedModelParameters

logger.enable('solarmed_modeling.solar_med.utils')

dfs_mod, stats = evaluate_model(
    df=df, sample_rate = sample_rate_numeric,
    model_params = ModelParameters(), 
    fixed_model_params = FixedModelParameters(),
    alternatives_to_eval = ["constant-water-props"],
)

fig = experimental_results_plot(plot_config, df, df_comp=dfs_mod[0], vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)
